## Load data

In [1]:
from google.colab import drive

drive.mount('/content/drive') #allow all access to Drive

Mounted at /content/drive


In [2]:
import os

#list files and directories in your Google Drive root
os.listdir('/content/drive/My Drive/')
os.listdir('/content/drive/My Drive/BTTAI Group 46 - PREPARED/')

['PREPARED: AI Studio Challenge Project Overview Deck shared.pptx',
 'Los Angeles Hazard Data.gsheet',
 'NRI_metadata_March2023.docx',
 'NRI_metadata_March2023.xml',
 'NRIDataDictionary.csv',
 'NRI_HazardInfo.csv',
 'NRI_Table_CensusTracts_California.gsheet',
 'Project Scope & Deliverables.gdoc',
 'Team Alliance.gdoc',
 'Archive',
 'Team Progress Summary.gdoc',
 'Data type .gdoc',
 'PREPARED - Sample Product Grid.xlsx',
 'Meeting Notes & Agendas (October).gdoc',
 'PREPARED AI Project Presentation.gslides',
 'Team_26_Project_Sample_Intro_to_RAG.ipynb']

In [3]:
import gspread
from google.colab import auth
from google.auth import default

#authenticate and create a client to interact with Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [4]:
#pip install descartes

In [5]:
import matplotlib.pyplot as plt
import pandas as pd
#import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
from gspread_dataframe import get_as_dataframe
from gspread_dataframe import set_with_dataframe

%matplotlib inline

In [6]:
spreadsheet = gc.open('Los Angeles Hazard Data')

#list all sheet names
worksheet_names = [worksheet.title for worksheet in spreadsheet.worksheets()]
print(*worksheet_names, sep='\n')

Sheet1
Wildfire_RCP45
Historic_Earthquakes
Tsunami_Tide_Gauges
Extreme_Heat
Los_Angeles_County_CVA_Social_Sensitivity_Index
Extreme_Precipitation_Low_Emissions_RCP45
LACoFD_Historic_Fire_Perimeters_(Feature_Layer)
LACoFD_Fire_Hazard_Severity_Zones_%E2%80%93_SRA_(Feature_Layer)
Social_Vulnerability_Index_2020_3398216233094230935
EJSM_Scores
Wildfire_RCP_8.5
LACoFD_Fire_Hazard_Severity_Zones_%E2%80%93_LRA_(Feature_Layer)
Extreme_Precipitation_Low_Emissions_RCP_8.5
Extreme_Heat_Low_Emissions_RCP_45


In [7]:
worksheet = spreadsheet.worksheet('Wildfire_RCP45')

#load the sheet into a DataFrame
wildfire_rcp45 = get_as_dataframe(worksheet)

In [9]:
worksheet = spreadsheet.worksheet('Tsunami_Tide_Gauges')
tsunami = get_as_dataframe(worksheet)

### Clean Tsunami Data

In [33]:
tsunami.head()

,X,Y,OBJECTID,source,ext_id,cat1,cat2,org_name,Name,post_id,link,use_type,latitude,longitude,date_updated,POINT_X,POINT_Y
0,-118.27286,33.71994,46928,HSIP Freedom Tsunami_Tide_Gauges,9410660,Environment,Tsunami Tide Gauges,National Oceanic and Atmospheric Administration,"Los Angeles, Outer Harbor",49389,http://egis3.lacounty.gov/lms/?p=49389,publish,33.71994,-118.27286,2010/11/01 19:50:56+00,6.478690e+06,1.720572e+06
1,-118.50000,34.00833,46929,HSIP Freedom Tsunami_Tide_Gauges,9410840,Environment,Tsunami Tide Gauges,National Oceanic and Atmospheric Administration,"Santa Monica, Pacific Ocean",49390,http://egis3.lacounty.gov/lms/?p=49390,publish,34.00833,-118.50000,2010/11/01 19:50:56+00,6.410135e+06,1.825789e+06


In [34]:
#drop rows where all elements are NaN
tsunami.dropna(how='all', inplace=True)
tsunami.head()

,X,Y,OBJECTID,source,ext_id,cat1,cat2,org_name,Name,post_id,link,use_type,latitude,longitude,date_updated,POINT_X,POINT_Y
0,-118.27286,33.71994,46928,HSIP Freedom Tsunami_Tide_Gauges,9410660,Environment,Tsunami Tide Gauges,National Oceanic and Atmospheric Administration,"Los Angeles, Outer Harbor",49389,http://egis3.lacounty.gov/lms/?p=49389,publish,33.71994,-118.27286,2010/11/01 19:50:56+00,6.478690e+06,1.720572e+06
1,-118.50000,34.00833,46929,HSIP Freedom Tsunami_Tide_Gauges,9410840,Environment,Tsunami Tide Gauges,National Oceanic and Atmospheric Administration,"Santa Monica, Pacific Ocean",49390,http://egis3.lacounty.gov/lms/?p=49390,publish,34.00833,-118.50000,2010/11/01 19:50:56+00,6.410135e+06,1.825789e+06


In [35]:
tsunami.dropna(how='all', axis=1, inplace=True)
print(tsunami.shape)
tsunami.head()

(2, 17)


,X,Y,OBJECTID,source,ext_id,cat1,cat2,org_name,Name,post_id,link,use_type,latitude,longitude,date_updated,POINT_X,POINT_Y
0,-118.27286,33.71994,46928,HSIP Freedom Tsunami_Tide_Gauges,9410660,Environment,Tsunami Tide Gauges,National Oceanic and Atmospheric Administration,"Los Angeles, Outer Harbor",49389,http://egis3.lacounty.gov/lms/?p=49389,publish,33.71994,-118.27286,2010/11/01 19:50:56+00,6.478690e+06,1.720572e+06
1,-118.50000,34.00833,46929,HSIP Freedom Tsunami_Tide_Gauges,9410840,Environment,Tsunami Tide Gauges,National Oceanic and Atmospheric Administration,"Santa Monica, Pacific Ocean",49390,http://egis3.lacounty.gov/lms/?p=49390,publish,34.00833,-118.50000,2010/11/01 19:50:56+00,6.410135e+06,1.825789e+06


In [36]:
# Export the tsunami data to google sheet
worksheet = spreadsheet.worksheet('Tsunami_Tide_Gauges')

# Convert the dataframe to a list of lists, including the column headers
data_to_export = [tsunami.columns.tolist()] + tsunami.values.tolist()

# Clear existing data in the worksheet
worksheet.clear()

# Update the worksheet with the data, starting at cell A1
worksheet.update('A1', data_to_export)

<ipython-input-36-1f564246276f>:11: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A1', data_to_export)


{'spreadsheetId': '1ZM93dk0vgK_rr5tjfWkGFmfsT9hqjUnan__Rnfv35_Y',
 'updatedRange': 'Tsunami_Tide_Gauges!A1:Q3',
 'updatedRows': 3,
 'updatedColumns': 17,
 'updatedCells': 51}

In [37]:
row_count = len(data_to_export)  # Number of rows with data
col_count = len(tsunami.columns) # Number of columns with data
worksheet.resize(row_count, col_count)

{'spreadsheetId': '1ZM93dk0vgK_rr5tjfWkGFmfsT9hqjUnan__Rnfv35_Y',
 'replies': [{}]}

In [38]:
tsunami = get_as_dataframe(worksheet)
print(tsunami.shape)
tsunami.head()

(2, 17)


,X,Y,OBJECTID,source,ext_id,cat1,cat2,org_name,Name,post_id,link,use_type,latitude,longitude,date_updated,POINT_X,POINT_Y
0,-118.27286,33.71994,46928,HSIP Freedom Tsunami_Tide_Gauges,9410660,Environment,Tsunami Tide Gauges,National Oceanic and Atmospheric Administration,"Los Angeles, Outer Harbor",49389,http://egis3.lacounty.gov/lms/?p=49389,publish,33.71994,-118.27286,2010/11/01 19:50:56+00,6.478690e+06,1.720572e+06
1,-118.50000,34.00833,46929,HSIP Freedom Tsunami_Tide_Gauges,9410840,Environment,Tsunami Tide Gauges,National Oceanic and Atmospheric Administration,"Santa Monica, Pacific Ocean",49390,http://egis3.lacounty.gov/lms/?p=49390,publish,34.00833,-118.50000,2010/11/01 19:50:56+00,6.410135e+06,1.825789e+06


### Clean Wild Fire 45 Data

In [39]:
print(wildfire_rcp45.shape)
wildfire_rcp45.tail()

(335, 14)


,OBJECTID,Geo_UID,Grid_Num,longitude,latitude,Baseline,F_2050,F_2080,Shape_Length,Shape_Area,Mid_Cent,Late_Cent,Shape__Area,Shape__Length
330,331,34.8125-118.6875,16879,-118.6875,34.8125,16.4358,-1.53,-2.77,0.136267,0.000352,14.906,13.666,5.315304e+06,15442.659886
331,332,34.8125-118.75,16700,-118.7500,34.8125,17.5342,-1.56,-0.55,0.136090,0.000347,15.974,16.984,5.235667e+06,15418.681383
332,333,34.8125-118.8125,16521,-118.8125,34.8125,18.0000,0.48,-0.95,0.135584,0.000334,18.480,17.050,5.040771e+06,15350.127670
333,334,34.8125-118.875,16342,-118.8750,34.8125,22.7900,0.79,-0.90,0.093408,0.000211,23.580,21.890,3.187506e+06,10648.926533
334,335,34.8125-118.9375,16163,-118.9375,34.8125,27.6483,0.78,0.63,0.034937,0.000057,28.428,28.278,8.558841e+05,4120.800765


In [40]:
#drop rows where all elements are NaN
wildfire_rcp45.dropna(how='all', inplace=True)
wildfire_rcp45.dropna(how='all', axis=1, inplace=True)
wildfire_rcp45.shape

(335, 14)

In [41]:
wildfire_rcp45.isna().sum()

,0
OBJECTID,0
Geo_UID,0
Grid_Num,0
longitude,0
latitude,0
Baseline,0
F_2050,0
F_2080,0
Shape_Length,0
Shape_Area,0


In [42]:
worksheet = spreadsheet.worksheet('Wildfire_RCP45')

# Convert the dataframe to a list of lists, including the column headers
data_to_export = [wildfire_rcp45.columns.tolist()] + wildfire_rcp45.values.tolist()

# Clear existing data in the worksheet
worksheet.clear()

# Update the worksheet with the data, starting at cell A1
worksheet.update('A1', data_to_export)

row_count = len(data_to_export)  # Number of rows with data
col_count = len(wildfire_rcp45.columns) # Number of columns with data
worksheet.resize(row_count, col_count)

wildfire_rcp45 = get_as_dataframe(worksheet)
print(wildfire_rcp45.shape)
wildfire_rcp45.head()

<ipython-input-42-3b26cb1f02ab>:10: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A1', data_to_export)


(335, 14)


,OBJECTID,Geo_UID,Grid_Num,longitude,latitude,Baseline,F_2050,F_2080,Shape_Length,Shape_Area,Mid_Cent,Late_Cent,Shape__Area,Shape__Length
0,1,32.75-118.4375,17562,-118.4375,32.7500,2.9133,-1.34,-1.97,0.110417,3.124653e-04,1.573,0.943,4.606830e+06,12910.061310
1,2,32.8125-118.375,17742,-118.3750,32.8125,4.2058,-1.87,-2.67,0.121772,5.804570e-04,2.336,1.536,8.559597e+06,14604.018687
2,3,32.8125-118.4375,17563,-118.4375,32.8125,19.1550,-6.92,-10.60,0.225823,3.189656e-03,12.235,8.555,4.704413e+07,27497.192826
3,4,32.8125-118.5,17384,-118.5000,32.8125,12.5058,-4.22,-6.74,0.218413,2.354185e-03,8.286,5.766,3.472695e+07,26584.168298
4,5,32.8125-118.5625,17205,-118.5625,32.8125,2.8242,-1.05,-1.66,0.005118,9.255045e-07,1.774,1.164,1.365556e+04,598.439164


### Clean Extreme Heat Data

In [13]:
worksheet = spreadsheet.worksheet('Extreme_Heat')
extremeHeat = get_as_dataframe(worksheet)

In [14]:
print(extremeHeat.shape)
extremeHeat.tail()

(323, 16)


,OBJECTID,Geo_UID,Grid_Num,longitude,latitude,Baseline,F_2050,F_2080,Temp_Exposure,Temp_Exposure_2080,Shape_Length,Shape_Area,Temp_Mid,Temp_Late,Shape__Area,Shape__Length
318,319,33.375-118.5625,NaN,-118.5625,33.3750,81.1940,3.7296,6.4027,Low,Low,0.145421,0.000574,84.924,87.597,8.517982e+06,16695.097342
319,320,33.375-118.625,NaN,-118.6250,33.3750,80.1527,3.8718,6.5595,Low,Low,0.009241,0.000002,84.024,86.712,3.598836e+04,1065.131392
320,321,33.4375-118.5,NaN,-118.5000,33.4375,81.9320,3.7793,6.5195,Low,Low,0.092838,0.000318,85.711,88.452,4.715637e+06,10740.541190
321,322,33.4375-118.5625,NaN,-118.5625,33.4375,80.5100,3.8945,6.6695,Low,Low,0.183046,0.001878,84.405,87.180,2.789538e+07,21844.194742
322,323,33.4375-118.625,NaN,-118.6250,33.4375,80.0600,3.9673,6.6942,Low,Low,0.141991,0.000963,84.027,86.754,1.429894e+07,17403.192501


In [15]:
#drop rows where all elements are NaN
extremeHeat.dropna(how='all', inplace=True)
extremeHeat.dropna(how='all', axis=1, inplace=True)
extremeHeat.shape

(323, 16)

In [16]:
extremeHeat.isna().sum()

,0
OBJECTID,0
Geo_UID,0
Grid_Num,14
longitude,0
latitude,0
Baseline,0
F_2050,0
F_2080,0
Temp_Exposure,0
Temp_Exposure_2080,0


In [17]:
print(extremeHeat['Grid_Num'].unique())
# Not sure what the values in this column represent, so let's keep it for now.
extremeHeat.fillna(0.0, inplace=True)

[437. 413. 389. 365. 341. 317. 438. 414. 390. 366. 342. 318. 463. 439.
 415. 391. 367. 343. 319. 488. 464. 440. 416. 392. 368. 344. 320. 296.
 561. 537. 513. 489. 465. 441. 417. 393. 369. 345. 321. 297. 586. 562.
 538. 514. 490. 466. 442. 418. 394. 370. 346. 322. 298. 274. 250. 226.
 202. 178. 154. 130. 106. 587. 563. 539. 515. 491. 467. 443. 419. 395.
 371. 347. 323. 299. 275. 251. 227. 203. 179. 155. 131. 107. 612. 588.
 564. 540. 516. 492. 468. 444. 420. 396. 372. 348. 324. 300. 276. 252.
 228. 204. 180. 156. 613. 589. 565. 541. 517. 493. 469. 445. 421. 397.
 373. 349. 325. 301. 277. 253. 229. 614. 590. 566. 542. 518. 494. 470.
 446. 422. 398. 374. 350. 326. 302. 278. 254. 230. 615. 591. 567. 543.
 519. 495. 471. 447. 423. 399. 375. 351. 327. 303. 279. 255. 231. 616.
 592. 568. 544. 520. 496. 472. 448. 424. 400. 376. 352. 328. 304. 280.
 256. 232. 208. 617. 593. 569. 545. 521. 497. 473. 449. 425. 401. 377.
 353. 329. 305. 281. 257. 233. 209. 618. 594. 570. 546. 522. 498. 474.
 450. 

In [18]:
data_to_export = [extremeHeat.columns.tolist()] + extremeHeat.values.tolist()

worksheet.update('A1', data_to_export)

row_count = len(data_to_export)  # Number of rows with data
col_count = len(extremeHeat.columns) # Number of columns with data
worksheet.resize(row_count, col_count)

extremeHeat = get_as_dataframe(worksheet)
print(extremeHeat.shape)
extremeHeat.head()

<ipython-input-18-0f4876306f6e>:3: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A1', data_to_export)


(323, 16)


,OBJECTID,Geo_UID,Grid_Num,longitude,latitude,Baseline,F_2050,F_2080,Temp_Exposure,Temp_Exposure_2080,Shape_Length,Shape_Area,Temp_Mid,Temp_Late,Shape__Area,Shape__Length
0,1,33.6875-118.125,437,-118.1250,33.6875,88.0547,4.56,7.28,Low,Low,0.048139,0.000034,92.615,95.335,5.063772e+05,5828.516560
1,2,33.6875-118.1875,413,-118.1875,33.6875,88.0520,4.48,7.20,Low,Low,0.052413,0.000026,92.532,95.252,3.809470e+05,6472.321641
2,3,33.6875-118.25,389,-118.2500,33.6875,87.3500,4.36,7.06,Low,Low,0.335520,0.000413,91.710,94.410,6.149175e+06,40519.205927
3,4,33.6875-118.3125,365,-118.3125,33.6875,85.4807,4.13,6.94,Low,Low,0.459919,0.001902,89.611,92.421,2.834756e+07,56315.667081
4,5,33.6875-118.375,341,-118.3750,33.6875,83.5907,4.06,6.85,Low,Low,0.190133,0.001461,87.651,90.441,2.176775e+07,22769.934508


### Define Functions to Drop Empty Rows and Fill Null Values

In [10]:
def fill_na_based_on_dtype(df):
    # Fill NaN for float columns with 0.0
    float_columns = df.select_dtypes(include=['float64', 'float32']).columns
    df[float_columns] = df[float_columns].fillna(0.0)

    # Fill NaN for string columns with ''
    string_columns = df.select_dtypes(include=['object']).columns
    df[string_columns] = df[string_columns].fillna('')

    return df

In [11]:
def clean_rows_cols(df):
    # Drop empty rows and columns
    df.dropna(how='all', inplace=True)
    df.dropna(how='all', axis=1, inplace=True)

In [20]:
def export_to_google_sheet(df):
    data_to_export = [df.columns.tolist()] + df.values.tolist()

    # Clear existing data in the worksheet
    worksheet.clear()

    # Resize and update the worksheet with the cleaned data
    worksheet.resize(len(data_to_export), len(df.columns))
    worksheet.update(range_name='A1', values=data_to_export)

### Clean Earthquake Data

In [22]:
worksheet = spreadsheet.worksheet('Historic_Earthquakes')
earthquake = get_as_dataframe(worksheet)

In [23]:
print(earthquake.shape)
earthquake.head()

(104, 33)


,X,Y,OBJECTID,source,ext_id,cat1,cat2,cat3,org_name,Name,...,use_type,latitude,longitude,date_updated,email,dis_status,POINT_X,POINT_Y,Unnamed: 31,Unnamed: 32
0,-118.0,34.5,13664.0,HSIP Freedom Earthquakes,13.0,Environment,Historic Earthquakes,NaN,US Geological Survey (USGS),Earthquake (1770),...,publish,34.5,-118.0,2010/11/01 19:50:56+00,NaN,NaN,6.561670e+06,2.004340e+06,NaN,NaN
1,-118.0,34.5,13665.0,HSIP Freedom Earthquakes,31.0,Environment,Historic Earthquakes,NaN,US Geological Survey (USGS),Earthquake (1803),...,publish,34.5,-118.0,2010/11/01 19:50:56+00,NaN,NaN,6.561670e+06,2.004340e+06,NaN,NaN
2,-118.1,34.1,13666.0,HSIP Freedom Earthquakes,113.0,Environment,Historic Earthquakes,NaN,US Geological Survey (USGS),Earthquake (1855),...,publish,34.1,-118.1,2010/11/01 19:50:56+00,NaN,NaN,6.531396e+06,1.858787e+06,NaN,NaN
3,-118.0,34.0,13667.0,HSIP Freedom Earthquakes,231.0,Environment,Historic Earthquakes,NaN,US Geological Survey (USGS),Earthquake (1878),...,publish,34.0,-118.0,2010/11/01 19:50:56+00,NaN,NaN,6.561670e+06,1.822380e+06,NaN,NaN
4,-117.9,34.1,13668.0,HSIP Freedom Earthquakes,317.0,Environment,Historic Earthquakes,NaN,US Geological Survey (USGS),Earthquake (1889),...,publish,34.1,-117.9,2010/11/01 19:50:56+00,NaN,NaN,6.591944e+06,1.858787e+06,NaN,NaN


In [24]:
clean_rows_cols(earthquake)
print(earthquake.shape)
earthquake.head()

(102, 18)


,X,Y,OBJECTID,source,ext_id,cat1,cat2,org_name,Name,info2,post_id,link,use_type,latitude,longitude,date_updated,POINT_X,POINT_Y
0,-118.0,34.5,13664.0,HSIP Freedom Earthquakes,13.0,Environment,Historic Earthquakes,US Geological Survey (USGS),Earthquake (1770),-9999.0,15805.0,http://egis3.lacounty.gov/lms/?p=15805,publish,34.5,-118.0,2010/11/01 19:50:56+00,6.561670e+06,2.004340e+06
1,-118.0,34.5,13665.0,HSIP Freedom Earthquakes,31.0,Environment,Historic Earthquakes,US Geological Survey (USGS),Earthquake (1803),-9999.0,15806.0,http://egis3.lacounty.gov/lms/?p=15806,publish,34.5,-118.0,2010/11/01 19:50:56+00,6.561670e+06,2.004340e+06
2,-118.1,34.1,13666.0,HSIP Freedom Earthquakes,113.0,Environment,Historic Earthquakes,US Geological Survey (USGS),Earthquake (1855),6.0,15807.0,http://egis3.lacounty.gov/lms/?p=15807,publish,34.1,-118.1,2010/11/01 19:50:56+00,6.531396e+06,1.858787e+06
3,-118.0,34.0,13667.0,HSIP Freedom Earthquakes,231.0,Environment,Historic Earthquakes,US Geological Survey (USGS),Earthquake (1878),-9999.0,15808.0,http://egis3.lacounty.gov/lms/?p=15808,publish,34.0,-118.0,2010/11/01 19:50:56+00,6.561670e+06,1.822380e+06
4,-117.9,34.1,13668.0,HSIP Freedom Earthquakes,317.0,Environment,Historic Earthquakes,US Geological Survey (USGS),Earthquake (1889),5.2,15809.0,http://egis3.lacounty.gov/lms/?p=15809,publish,34.1,-117.9,2010/11/01 19:50:56+00,6.591944e+06,1.858787e+06


In [25]:
earthquake.isna().sum()

,0
X,0
Y,0
OBJECTID,0
source,0
ext_id,0
cat1,0
cat2,0
org_name,0
Name,0
info2,0


In [26]:
export_to_google_sheet(earthquake)

### Clean Social Sensitivity Index Data

In [27]:
worksheet = spreadsheet.worksheet('Los_Angeles_County_CVA_Social_Sensitivity_Index')
Social_Sensitivity_Index = get_as_dataframe(worksheet)

In [28]:
print(Social_Sensitivity_Index.shape)
Social_Sensitivity_Index.head()

(2329, 48)


,Census_Tract,County,CSA_Type,CSA_Label,DRP_Planning_Area,DPH_Service_Planning_Area,City_of_Los_Angeles_CPA,Population,Children,Older_Adults,...,NHOPI,Other_Race,Two_or_More_Races,SoVI_Score,SoVI_Thirds,ObjectId,Shape__Area,Shape__Length,Unnamed: 46,Unnamed: 47
0,Census Tract 5709.02,Los Angeles County,City,City of Lakewood,Gateway Planning Area,East,NaN,3765.0,27.8,14.0,...,0.0,0.4,8.0,2.25,3.0,1.0,1.536760e+06,5208.283880,NaN,NaN
1,Census Tract 5715.02,Los Angeles County,City,City of Long Beach,Gateway Planning Area,South Bay,NaN,4700.0,24.6,14.0,...,0.1,0.6,1.3,-1.77,1.0,2.0,2.001416e+06,5938.830961,NaN,NaN
2,Census Tract 9106.05,Los Angeles County,City,City of Palmdale,Antelope Valley Planning Area,Antelope Valley,NaN,4710.0,38.4,7.0,...,0.0,0.2,2.5,2.39,3.0,3.0,1.996430e+06,5983.046762,NaN,NaN
3,Census Tract 9107.12,Los Angeles County,City,City of Palmdale,Antelope Valley Planning Area,Antelope Valley,NaN,2924.0,30.8,12.2,...,0.0,0.0,1.6,-1.05,2.0,4.0,3.107240e+06,7109.522437,NaN,NaN
4,Census Tract 9107.15,Los Angeles County,City,City of Palmdale,Antelope Valley Planning Area,Antelope Valley,NaN,6494.0,33.7,6.9,...,0.0,0.0,0.4,2.47,3.0,5.0,2.925328e+06,7944.828622,NaN,NaN


In [29]:
clean_rows_cols(Social_Sensitivity_Index)

In [30]:
print(Social_Sensitivity_Index.shape)
Social_Sensitivity_Index.head()

(2327, 46)


,Census_Tract,County,CSA_Type,CSA_Label,DRP_Planning_Area,DPH_Service_Planning_Area,City_of_Los_Angeles_CPA,Population,Children,Older_Adults,...,Native_Tribal,Asian,NHOPI,Other_Race,Two_or_More_Races,SoVI_Score,SoVI_Thirds,ObjectId,Shape__Area,Shape__Length
0,Census Tract 5709.02,Los Angeles County,City,City of Lakewood,Gateway Planning Area,East,NaN,3765.0,27.8,14.0,...,0.0,10.9,0.0,0.4,8.0,2.25,3.0,1.0,1.536760e+06,5208.283880
1,Census Tract 5715.02,Los Angeles County,City,City of Long Beach,Gateway Planning Area,South Bay,NaN,4700.0,24.6,14.0,...,0.0,21.7,0.1,0.6,1.3,-1.77,1.0,2.0,2.001416e+06,5938.830961
2,Census Tract 9106.05,Los Angeles County,City,City of Palmdale,Antelope Valley Planning Area,Antelope Valley,NaN,4710.0,38.4,7.0,...,0.0,5.3,0.0,0.2,2.5,2.39,3.0,3.0,1.996430e+06,5983.046762
3,Census Tract 9107.12,Los Angeles County,City,City of Palmdale,Antelope Valley Planning Area,Antelope Valley,NaN,2924.0,30.8,12.2,...,0.0,1.3,0.0,0.0,1.6,-1.05,2.0,4.0,3.107240e+06,7109.522437
4,Census Tract 9107.15,Los Angeles County,City,City of Palmdale,Antelope Valley Planning Area,Antelope Valley,NaN,6494.0,33.7,6.9,...,0.3,1.2,0.0,0.0,0.4,2.47,3.0,5.0,2.925328e+06,7944.828622


In [32]:
Social_Sensitivity_Index.isna().sum()

,0
Census_Tract,0
County,0
CSA_Type,0
CSA_Label,0
DRP_Planning_Area,0
DPH_Service_Planning_Area,0
City_of_Los_Angeles_CPA,1333
Population,0
Children,0
Older_Adults,0


In [34]:
# Community Planning Areas (CPAs)
# The City of Los Angeles has 36 CPAs, including the Port of Los Angeles.
# The names of these areas are usually made up of hyphenated groups of neighborhoods and communities.
print(Social_Sensitivity_Index['City_of_Los_Angeles_CPA'].nunique())
Social_Sensitivity_Index['City_of_Los_Angeles_CPA'].unique()
# Each row is connected to a Census Tract, so let's not delete them.

37


array([nan, 'Arleta - Pacoima', 'Sylmar', 'San Pedro',
       'Sherman Oaks - Studio City - Toluca Lake - Cahuenga Pass',
       'Northeast Los Angeles',
       'Silver Lake - Echo Park - Elysian Valley', 'Hollywood',
       'Sunland - Tujunga - Lake View Terrace - Shadow Hills - East La Tuna Canyon',
       'Southeast Los Angeles', 'Sun Valley - La Tuna Canyon',
       'North Hollywood - Valley Village', 'Boyle Heights', 'Westlake',
       'Wilshire', 'South Los Angeles', 'Westchester - Playa del Rey',
       'Wilmington - Harbor City', 'Central City',
       'Canoga Park - Winnetka - Woodland Hills - West Hills',
       'Encino - Tarzana', 'West Los Angeles',
       'Granada Hills - Knollwood', 'Chatsworth - Porter Ranch',
       'Northridge', 'Mission Hills - Panorama City - North Hills',
       'Van Nuys - North Sherman Oaks', 'Reseda - West Van Nuys',
       'Harbor Gateway', 'Westwood', 'Palms - Mar Vista - Del Rey',
       'West Adams - Baldwin Hills - Leimert', 'Venice',
      

In [36]:
fill_na_based_on_dtype(Social_Sensitivity_Index)

,Census_Tract,County,CSA_Type,CSA_Label,DRP_Planning_Area,DPH_Service_Planning_Area,City_of_Los_Angeles_CPA,Population,Children,Older_Adults,...,Native_Tribal,Asian,NHOPI,Other_Race,Two_or_More_Races,SoVI_Score,SoVI_Thirds,ObjectId,Shape__Area,Shape__Length
0,Census Tract 5709.02,Los Angeles County,City,City of Lakewood,Gateway Planning Area,East,,3765.0,27.8,14.0,...,0.0,10.9,0.0,0.4,8.0,2.25,3.0,1.0,1.536760e+06,5208.283880
1,Census Tract 5715.02,Los Angeles County,City,City of Long Beach,Gateway Planning Area,South Bay,,4700.0,24.6,14.0,...,0.0,21.7,0.1,0.6,1.3,-1.77,1.0,2.0,2.001416e+06,5938.830961
2,Census Tract 9106.05,Los Angeles County,City,City of Palmdale,Antelope Valley Planning Area,Antelope Valley,,4710.0,38.4,7.0,...,0.0,5.3,0.0,0.2,2.5,2.39,3.0,3.0,1.996430e+06,5983.046762
3,Census Tract 9107.12,Los Angeles County,City,City of Palmdale,Antelope Valley Planning Area,Antelope Valley,,2924.0,30.8,12.2,...,0.0,1.3,0.0,0.0,1.6,-1.05,2.0,4.0,3.107240e+06,7109.522437
4,Census Tract 9107.15,Los Angeles County,City,City of Palmdale,Antelope Valley Planning Area,Antelope Valley,,6494.0,33.7,6.9,...,0.3,1.2,0.0,0.0,0.4,2.47,3.0,5.0,2.925328e+06,7944.828622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2322,Census Tract 6010.02,Los Angeles County,City,City of Inglewood,South Bay Planning Area,South Bay,,5099.0,28.9,13.4,...,3.2,1.7,0.0,1.8,2.5,3.23,3.0,2323.0,9.198689e+05,4108.691967
2323,Census Tract 9005.01,Los Angeles County,City,City of Lancaster,Antelope Valley Planning Area,Antelope Valley,,6712.0,29.2,9.2,...,0.0,4.3,0.0,0.0,3.4,0.14,2.0,2324.0,4.619789e+06,8637.285412
2324,Census Tract 9200.29,Los Angeles County,City,City of Santa Clarita,Santa Clarita Valley Planning Area,San Fernando,,3956.0,28.4,9.4,...,0.0,6.9,0.1,0.0,2.4,-2.35,1.0,2325.0,3.313415e+06,10004.044231
2325,Census Tract 5420,Los Angeles County,City,City of Compton,Metro Planning Area,South,,4606.0,30.1,10.9,...,0.0,1.2,0.0,0.0,0.0,3.92,3.0,2326.0,1.436066e+06,5044.552153


In [37]:
export_to_google_sheet(Social_Sensitivity_Index)

### Clean Low Emissions RCP45 Data

In [38]:
worksheet = spreadsheet.worksheet('Extreme_Precipitation_Low_Emissions_RCP45')
Low_Emissions_RCP45 = get_as_dataframe(worksheet)

In [39]:
print(Low_Emissions_RCP45.shape)
Low_Emissions_RCP45.head()

(999, 26)


,OBJECTID,Geo_UID,longitude,latitude,Baseline,F_2050,F_2080,Shape_Length,Shape_Area,Mid_Cent,...,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,1.0,32.75-118.4375,-118.4375,32.7500,13.4009,0.3767,1.5012,0.110417,3.124653e-04,13.778,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,32.8125-118.375,-118.3750,32.8125,13.8268,0.2638,1.2594,0.121772,5.804570e-04,14.091,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,32.8125-118.4375,-118.4375,32.8125,14.6786,0.2402,1.2304,0.225823,3.189656e-03,14.919,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,32.8125-118.5,-118.5000,32.8125,15.7035,0.3480,1.3587,0.218413,2.354185e-03,16.052,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,32.8125-118.5625,-118.5625,32.8125,14.9741,0.4115,1.1637,0.005118,9.255058e-07,15.386,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
clean_rows_cols(Low_Emissions_RCP45)

In [43]:
print(Low_Emissions_RCP45.shape)
Low_Emissions_RCP45.isna().sum()

(335, 13)


,0
OBJECTID,0
Geo_UID,0
longitude,0
latitude,0
Baseline,0
F_2050,0
F_2080,0
Shape_Length,0
Shape_Area,0
Mid_Cent,0


In [44]:
export_to_google_sheet(Low_Emissions_RCP45)

### Clean Fire Perimeters Data

In [49]:
worksheet = spreadsheet.worksheet('LACoFD_Historic_Fire_Perimeters_(Feature_Layer)')
fire_perimeter = get_as_dataframe(worksheet)

In [50]:
print(fire_perimeter.shape)
clean_rows_cols(fire_perimeter)
print(fire_perimeter.shape)
fire_perimeter.isna().sum()

(999, 26)
(891, 8)


,0
OBJECTID_1,0
OBJECTID,0
YEAR,0
FIRE_NAME,0
GIS_ACRES,0
Shape_Leng,0
Shape__Area,0
Shape__Length,0


In [51]:
export_to_google_sheet(fire_perimeter)

### Clean Fire Hazard Severity Zones Data

In [53]:
worksheet = spreadsheet.worksheet('LACoFD_Fire_Hazard_Severity_Zones_%E2%80%93_SRA_(Feature_Layer)')
fire_zones = get_as_dataframe(worksheet)

In [54]:
print(fire_zones.shape)
clean_rows_cols(fire_zones)
print(fire_zones.shape)
fire_zones.isna().sum()

(1236, 12)
(1234, 10)


,0
FID,0
OBJECTID,0
SRA,0
HAZ_CODE,0
HAZ_CLASS,0
Shape_Leng,0
Revised,0
Shape__Area,0
Shape__Length,0
GlobalID,0


In [55]:
export_to_google_sheet(fire_zones)

### Clean Social Vulnerability Index

In [58]:
worksheet = spreadsheet.worksheet('Social_Vulnerability_Index_2020_3398216233094230935')
vulnerability_index = get_as_dataframe(worksheet)

In [59]:
print(vulnerability_index.shape)
clean_rows_cols(vulnerability_index)
print(vulnerability_index.shape)
vulnerability_index.isna().sum()

(2497, 28)
(2495, 26)


,0
OBJECTID_1,0
Tract 2020,0
Tract Label,0
FIPS,0
Location,0
Area in Square Miles,0
"Population estimate, 2016-2020 ACS",0
"Population estimate MOE, 2016-2020 ACS",0
"Housing units estimate, 2016-2020 ACS",0
"Housing units estimate MOE, 2016-2020 ACS",0


In [61]:
vulnerability_index['Level of Vulnerability'].unique()
# Each row is linked to a Census Tract, so let's keep them for now.

array(['Medium to High', 'Low', 'High', 'Low to Medium', nan],
      dtype=object)

In [63]:
fill_na_based_on_dtype(vulnerability_index)
export_to_google_sheet(vulnerability_index)

### Clean EJSM Scores

In [65]:
worksheet = spreadsheet.worksheet('EJSM_Scores')
EJSM = get_as_dataframe(worksheet)

In [66]:
print(EJSM.shape)
clean_rows_cols(EJSM)
print(EJSM.shape)
EJSM.isna().sum()

(2345, 11)
(2343, 9)


,0
OBJECTID,0
Tract_1,0
CIscore,0
HazScore,0
HealthScore,0
SVscore,0
CCVscore,0
Shape__Area,0
Shape__Length,0


In [67]:
export_to_google_sheet(EJSM)

### Clean Wildfire_RCP_8.5 Data

In [68]:
worksheet = spreadsheet.worksheet('Wildfire_RCP_8.5')
wildfire_rcp85 = get_as_dataframe(worksheet)

In [69]:
print(wildfire_rcp85.shape)
clean_rows_cols(wildfire_rcp85)
print(wildfire_rcp85.shape)
wildfire_rcp85.isna().sum()

(999, 26)
(337, 18)


,0
OBJECTID,0
Geo_UID,0
Grid_Num,0
longitude,0
latitude,0
Baseline,0
F_2050,0
F_2080,0
Field_7,14
Wildfire_E,0


In [71]:
wildfire_rcp85['Null_'].unique()
# Not sure if this is boolean, but doesn't seem important, so we decide to delete this column.

array([nan,  1.])

In [72]:
wildfire_rcp85.drop(columns=['Null_'], inplace=True)

In [73]:
wildfire_rcp85['Field_7'].unique()
# This column also doesn't seem important, so we decide to delete it.

array([ 0., nan])

In [74]:
wildfire_rcp85.drop(columns=['Field_7'], inplace=True)

In [75]:
wildfire_rcp85.isna().sum()

,0
OBJECTID,0
Geo_UID,0
Grid_Num,0
longitude,0
latitude,0
Baseline,0
F_2050,0
F_2080,0
Wildfire_E,0
Wildfire_1,0


In [76]:
export_to_google_sheet(wildfire_rcp85)

### Clean Severity_Zones_LRA Data

In [77]:
worksheet = spreadsheet.worksheet('LACoFD_Fire_Hazard_Severity_Zones_%E2%80%93_LRA_(Feature_Layer)')
lra = get_as_dataframe(worksheet)

In [78]:
print(lra.shape)
clean_rows_cols(lra)
print(lra.shape)
lra.isna().sum()

(999, 26)
(161, 14)


,0
FID,0
OBJECTID_1,0
OBJECTID,0
FID_c19fhs,0
HAZ_CODE,0
HAZ_CLASS,0
SRA,0
INCORP,95
VH_REC,157
Shape_Leng,0


In [79]:
lra['INCORP'].unique()
# Unique value is Y for Yes? We'll fill the null rows with 'N'

array([nan, 'Y'], dtype=object)

In [80]:
lra['INCORP'] = lra['INCORP'].fillna('N')
lra['INCORP'].unique()

array(['N', 'Y'], dtype=object)

In [82]:
lra['VH_REC'].unique()
fill_na_based_on_dtype(lra)

,FID,OBJECTID_1,OBJECTID,FID_c19fhs,HAZ_CODE,HAZ_CLASS,SRA,INCORP,VH_REC,Shape_Leng,Revised,Shape__Area,Shape__Length,GlobalID
0,1.0,1.0,1.0,0.0,3.0,Very High,LRA,N,,5415.460854,2008/01/01 00:00:00+00,1.459306e+07,17768.391203,4b489a43-0a77-47c8-af97-8bb59468ccc3
1,2.0,2.0,2.0,1.0,3.0,Very High,LRA,N,,2802.070818,2008/01/01 00:00:00+00,4.560774e+06,9193.548030,11362477-e50d-4763-b8b2-45b3f7b064f3
2,3.0,3.0,3.0,2.0,3.0,Very High,LRA,N,,802.128633,2008/01/01 00:00:00+00,3.473848e+05,2631.672402,2767eb44-ff54-4da6-aed8-344af9c525aa
3,4.0,4.0,4.0,3.0,3.0,Very High,LRA,N,,1096.587821,2008/01/01 00:00:00+00,4.391797e+05,3597.732571,fc1457c8-533f-4e89-bda2-63f4afc812ac
4,5.0,5.0,5.0,4.0,3.0,Very High,LRA,N,,59225.977459,2008/01/01 00:00:00+00,1.009652e+08,194312.804614,331817d8-acf4-4a9a-8ba4-ced47ec039b5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,157.0,157.0,157.0,113.0,3.0,Very High,LRA,Y,,669.639380,2008/01/01 00:00:00+00,2.009473e+01,2197.153951,9e006805-a84f-4bf5-84f5-7648577ebff4
157,158.0,158.0,158.0,114.0,3.0,Very High,LRA,Y,,2499.457940,2008/01/01 00:00:00+00,6.065234e+01,8200.983048,1697d237-eb55-4bdf-9853-2e2fe92d3e9b
158,159.0,159.0,159.0,116.0,3.0,Very High,LRA,Y,,7579.565678,2008/01/01 00:00:00+00,2.469160e+02,24862.294350,bc96f83b-5459-44d2-b2ff-c44d9f2446ff
159,160.0,160.0,160.0,117.0,3.0,Very High,LRA,Y,,785.811901,2008/01/01 00:00:00+00,7.927734e+00,2578.157148,6afa75d2-44df-4b4b-830c-a93ada86da8f


In [83]:
export_to_google_sheet(lra)

### Clean Extreme_Precipitation_Low_Emissions_RCP_8.5

In [84]:
worksheet = spreadsheet.worksheet('Extreme_Precipitation_Low_Emissions_RCP_8.5')
em_rcp85 = get_as_dataframe(worksheet)

In [85]:
print(em_rcp85.shape)
clean_rows_cols(em_rcp85)
print(em_rcp85.shape)
em_rcp85.isna().sum()

(999, 26)
(323, 19)


,0
OBJECTID,0
Geo_UID,0
Grid_Num,14
longitude,0
latitude,0
Baseline,0
F_2050,0
F_2080,0
Baseline_Inches,14
F_2050_Inches,0


In [90]:
em_rcp85['Precip_Exposure'].unique()
em_rcp85.dropna(subset=['Precip_Exposure'], inplace=True)
em_rcp85.shape

(322, 19)

In [91]:
em_rcp85['Precip_Exposure'].unique()

array(['Medium', 'High', 'Low'], dtype=object)

In [104]:
average = em_rcp85['Baseline_Inches'].mean()
em_rcp85['Baseline_Inches'] = em_rcp85['Baseline_Inches'].fillna(average)

In [105]:
print(average)
em_rcp85['Baseline_Inches'].unique()

1.0335258298136647


array([0.9423071, 0.9753898, 0.9748465, 1.009689 , 0.9783425, 0.854437 ,
       1.077835 , 1.15422  , 1.002854 , 1.016146 , 1.090783 , 0.9799646,
       0.9647716, 1.068429 , 1.07361  , 1.001118 , 1.002748 , 1.000354 ,
       0.9455118, 1.001303 , 1.0715   , 1.122425 , 1.168886 , 1.105669 ,
       1.110079 , 1.080705 , 1.092067 , 0.9819685, 1.155555 , 1.219358 ,
       1.130579 , 1.178583 , 1.200122 , 1.103449 , 1.081622 , 1.172862 ,
       1.21785  , 1.21774  , 1.112634 , 1.032677 , 1.02365  , 1.117394 ,
       1.089819 , 1.062398 , 1.066197 , 1.101953 , 1.246469 , 1.154425 ,
       1.324382 , 1.272209 , 1.194327 , 1.187579 , 1.153933 , 1.081673 ,
       1.169736 , 1.126079 , 1.08022  , 1.095398 , 1.002319 , 0.8980039,
       0.8201851, 1.186378 , 1.221728 , 1.141098 , 1.100461 , 1.056323 ,
       1.109925 , 1.351386 , 1.26198  , 1.279047 , 1.290343 , 1.282862 ,
       1.332996 , 1.566398 , 1.603492 , 1.77335  , 1.554654 , 1.465024 ,
       1.524193 , 1.553925 , 1.459461 , 1.236504 , 

In [ ]:
fill_na_based_on_dtype(em_rcp85)

In [97]:
em_rcp85.isna().sum()

,0
OBJECTID,0
Geo_UID,0
Grid_Num,0
longitude,0
latitude,0
Baseline,0
F_2050,0
F_2080,0
Baseline_Inches,0
F_2050_Inches,0


In [99]:
export_to_google_sheet(em_rcp85)

### Clean Extreme_Heat_Low_Emissions_RCP_45

In [106]:
worksheet = spreadsheet.worksheet('Extreme_Heat_Low_Emissions_RCP_45')
heat_em_rcp45 = get_as_dataframe(worksheet)

In [107]:
print(heat_em_rcp45.shape)
clean_rows_cols(heat_em_rcp45)
print(heat_em_rcp45.shape)
heat_em_rcp45.isna().sum()

(999, 26)
(335, 13)


,0
OBJECTID,0
Geo_UID,0
longitude,0
latitude,0
Baseline,0
F_2050,0
F_2080,0
Shape_Length,0
Shape_Area,0
Mid_Cent,0


In [108]:
export_to_google_sheet(heat_em_rcp45)